# Démo DDPM

## Definition

Les diffusions modèles (ici [DDPM](https://arxiv.org/abs/2006.11239)) sont une branche d'algorithmes utilisés pour la génération de données.

![](https://learnopencv.com/wp-content/uploads/2023/01/diffusion-models-forwardbackward_process_ddpm.png)

Le principe est simple, il y a un premier processus de diffusion consistant à ajouter progressivement du bruit à la donnée. Ce processus de diffusion consiste en une chaine de Markov où il est possible d'obtenir la donnée à t + 1 (donc un peu plus bruitée) à partir de la donnée à un temps t :

![](./resources/diffusion_formula.png)

où les betas représentent la variance à un temps t, et sont croissants linéairement sur 1000 étapes de 1e-4 (beta 0) à 2e-2 (beta 1000).

De l'autre côté, nous avons une seconde chaine de Markov qui vise à de-bruiter l'image précédemment diffusée (bruitée) :

![](./resources/reverse_formula.png)

Regardons ça de plus près, mais d'abord des imports !




In [ ]:
from PIL import Image
import torch as th
from torch import nn
from torchvision.transforms.functional import to_tensor
import matplotlib.pyplot as plt
from tqdm import tqdm
from typing import Literal
import math

## Forward process - diffusion

In [ ]:
!wget https://media.cnn.com/api/v1/images/stellar/prod/190430171751-mona-lisa.jpg

x_0 = to_tensor(Image.open("./190430171751-mona-lisa.jpg"))

In [ ]:
T = 100
beta_1 = 1e-3
beta_t = 2e-1

betas = th.linspace(beta_1, beta_t, steps=T)

In [ ]:
def q_step(x_t_prev: th.Tensor, t: int) -> th.Tensor:
    z = th.randn_like(x_t_prev)
    return (1 - betas[t]) * x_t_prev + z * betas[t]

In [ ]:
x_1 = q_step(x_0, 1)

In [ ]:
x_0.size(), x_1.size()

In [ ]:
plt.imshow(x_0.permute(1, 2, 0))

In [ ]:
plt.imshow(x_1.permute(1, 2, 0))

In [ ]:
x_t_list = [x_0]

for t in tqdm(range(1, T)):
    x_t_list.append(q_step(x_t_list[-1], t))

In [ ]:
plt.imshow(x_t_list[25].permute(1, 2, 0))

In [ ]:
plt.imshow(x_t_list[40].permute(1, 2, 0))

In [ ]:
plt.imshow(x_t_list[-1].permute(1, 2, 0))

3 secondes pour "diffuser" une image, c'est trop ! Les auteurs proposent une simplification permettant d'obtenir n'import quel x (de t = 1 à t = T) à partir de la donnée originelle, le x à t = 0 :

![](https://miro.medium.com/v2/resize:fit:660/1*SRUnVsytTzuCWLvu7tA4gA.png)

Ce qui donne en code :

In [ ]:
alphas = 1 - betas
alphas_cum_prod = th.cumprod(alphas, dim=0)

In [ ]:
def q_sample(x_0: th.Tensor, t: int) -> th.Tensor:
    z = th.randn_like(x_0)
    return th.sqrt(alphas_cum_prod[t]) * x_0 + (1 - alphas_cum_prod[t]) * z

In [ ]:
x_10 = q_sample(x_0, 10)
plt.matshow(x_10.permute(1, 2, 0))

In [ ]:
x_30 = q_sample(x_0, 30)
plt.matshow(x_30.permute(1, 2, 0))

In [ ]:
x_70 = q_sample(x_0, 70)
plt.matshow(x_70.permute(1, 2, 0))

## Reverse process - denoising process

C'est ok pour le processus de diffusion, qu'en est-il pour le coeur du sujet : le processus inverse aka le de-bruitage ?

Il s'agit aussi d'une chaine de markov : à une étape t, il faut prédire la moyenne et la matrice de covariance qui ont servi à ajouter le bruit à l'étape précédente (pour passer de t - 1 à t) :

![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRCZCpFnGkY_TCOPvvy-W3rOBuKR-ZPTbx6Pg&usqp=CAU)

Le modèle reçoit deux paramètres :
- la donnée bruitée
- l'indice dans la chaine de markov

Les auteurs simplifient le tout (en rapport au processus de diffusion / bruitage amélioré) :
- on oublie la matrice de covariance du fait que la variance sera constante (matrice identité avec comme facteur beta)
- on ne prédit plus la moyenne de la distribution normale, mais directement le bruit

Le tout donne en algorithme :

![](https://huggingface.co/blog/assets/78_annotated-diffusion/training.png)

## U-Net architecture

Il nous faut une architecture de réseau de neurones qui puisse à partir d'une image, produire une image de mêmes dimensions mais dans un espace de canaux / pixels / couleurs différent. Ici l'espace à prédire pour les pixels sera le bruit qui a été ajouté.

L'architecture U-Net sera parfaite : elle a fait ses preuves dans le biomédical pour de la segmentation d'images (plus généralement passer dans un autre espace de couleurs / canaux / pixels). Elle consiste en deux parties :
- un encodeur
- un décodeur

Ces deux parties sont liées par le "milieu" (la connexion encodeur vers décodeur) mais aussi par des connexions directes entre les couches de l'encodeur et celles du décodeur :

![](https://miro.medium.com/v2/resize:fit:1400/1*f7YOaE4TWubwaFF7Z1fzNw.png)

### L'élément de base : la convolution

![](https://upload.wikimedia.org/wikipedia/commons/0/04/Convolution_arithmetic_-_Padding_strides.gif?20190413174630)

![](https://miro.medium.com/v2/resize:fit:640/1*ZCjPUFrB6eHPRi4eyP6aaA.gif)

Nous allons créer notre block (ou couche) de base comprenant :
- une convolution 3 x 3
- une activation : Mish
- une couche de normalisation : GroupNorm


In [ ]:
class ConvBlock(nn.Sequential):
    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        group_norm: int,
    ) -> None:
        super().__init__(
            nn.Conv2d(
                in_channels,
                out_channels,
                kernel_size=(3, 3),
                padding=(1, 1),
                stride=(1, 1),
            ),
            nn.Mish(),
            nn.GroupNorm(group_norm, out_channels),
        )

In [ ]:
block_1 = ConvBlock(3, 8, 4)

In [ ]:
x_30.size()

In [ ]:
x_30 = x_30.unsqueeze(0)
out = block_1(x_30)

In [ ]:
out.size()

### Up sample / down sample

Pour diminuer la taille de nos images latentes (celles entre les couches de l'encodeur) : on applique un pas de 2 pour nos convolutions.

Pour augmenter la taille des images latentes (celles entre les couches du décodeur) : des convolutions transposées à pas de 2.

![](https://miro.medium.com/v2/resize:fit:720/1*kOThnLR8Fge_AJcHrkR3dg.gif)

In [ ]:
class StrideConvBlock(nn.Sequential):
    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        norm_groups: int,
        mode: Literal["up", "down"],
    ) -> None:
        conv_constructors = {
            "up": nn.ConvTranspose2d,
            "down": nn.Conv2d,
        }

        super().__init__(
            conv_constructors[mode](
                in_channels,
                out_channels,
                kernel_size=(4, 4),
                padding=(1, 1),
                stride=(2, 2)
            ),
            nn.Mish(),
            nn.GroupNorm(norm_groups, out_channels),
        )

In [ ]:
down_block = StrideConvBlock(8, 16, 4, "down")

In [ ]:
out_2 = down_block(out)

In [ ]:
out.size(), out_2.size()

In [ ]:
up_block = StrideConvBlock(16, 8, 4, "up")

In [ ]:
out_3 = up_block(out_2)

In [ ]:
out_2.size(), out_3.size()

### Time embedding

Nous disposons maintenant des briques de base pour notre U-Net. Il ne manque plus qu'à intégrer le paramètre supplémentaire représentant l'indice de l'étape dans le processus de diffusion :
$$\mu_{\theta }(x_{t},t)$$

L'idée : "injecter" à chaque couche (ie chaque image intermédiaire / image latente) l'information du temps. Soucis : d'un coté (image) on a des valeurs continues, de l'autr coté (indice de l'étape) des valeurs discrètes.
La solution : des vecteurs d'embedding.

Les auteurs de DDPM utilisent un embedding sans paramètres (sans avoir besoin d'être entrainé) pour représenter un indice d'étape dans la chaine de markov : l'embedding sinusoïde aka le positional embedding (cf. Attention is all you need).

![](https://i.stack.imgur.com/67ADh.png)

Ce qui donne en code :




In [ ]:
TIME_EMBEDDING_SIZE = 16

In [ ]:
pos_emb = th.zeros(T, TIME_EMBEDDING_SIZE)
print(pos_emb.size())

position = th.arange(0, T).unsqueeze(1)
print(position.size())

div_term = th.exp(
    th.arange(0, TIME_EMBEDDING_SIZE, 2, dtype=th.float)
    * th.tensor(-math.log(10000.0) / T)
)
print(div_term.size())

pos_emb[:, 0::2] = th.sin(position.float() * div_term)
pos_emb[:, 1::2] = th.cos(position.float() * div_term)

In [ ]:
pos_emb[0:3]

Ok mais maintenant comment on l'injecte ? D'un côté j'ai une matrice carrée (même cubique si on compte les "couleurs" latentes) et de l'autre côté un seul vecteur ?

1. Projeter (avec des paramètres entrainables !) le vecteur d'embedding de temps dans l'espace des "couleurs" de l'image latente / intermédiaire
2. Ajouter ce vecteur à chaque pixel de l'image

En code :

In [ ]:
time_proj = nn.Sequential(
    nn.Linear(TIME_EMBEDDING_SIZE, 8),
    nn.Mish(),
    nn.Linear(8, 8)
)

In [ ]:
print("first block :")
print(block_1)

In [ ]:
block_2 = ConvBlock(8, 16, 8)

In [ ]:
print(block_2)

In [ ]:
x_30.size()

In [ ]:
t = th.randint(0, T, (x_30.size(0),))

In [ ]:
t.size()

In [ ]:
t_embedding = pos_emb[t]
print("emb", t_embedding.size())

t_projected = time_proj(t_embedding)
print("proj", t_projected.size())

t_projected_unsqueezed = t_projected[:, :, None, None]
print("proj-unsqueez", t_projected_unsqueezed.size())

out_1 = block_1(x_30)
print("out-1", out_1.size())

out_2 = out_1 + t_projected_unsqueezed
print("out-2", out_2.size())

out_3 = block_2(out_2)
print("out-3", out_3.size())

Il y a maintenant (presque) toutes les briques de base du U-Net, voici l'enchainement pour la partie encodeur :

In [ ]:
# encoder's layers definition

channels = [(8, 16), (16, 32), (32, 64)]
input_channels = 3
norm_groups = 4

encoder_time_to_channel_blocks = nn.ModuleList(
    nn.Sequential(
        nn.Linear(TIME_EMBEDDING_SIZE, c_i),
        nn.Mish(),
        nn.Linear(c_i, c_i)
    )
    for c_i, _ in channels
)

first_conv = ConvBlock(input_channels, channels[0][0], norm_groups)

encoder_conv_blocks = nn.ModuleList(
    nn.Sequential(
        ConvBlock(c_i, c_o, norm_groups),
        ConvBlock(c_o, c_o, norm_groups),
    )
    for c_i, c_o in channels
)

down_conv_blocks = nn.ModuleList(
    StrideConvBlock(c_o, c_o, norm_groups, "down")
    for _, c_o in channels
)

In [ ]:
# inputs definition

x = x_30.clone()
t = th.randint(0, T, (x_30.size(0),))

In [ ]:
# encoder forward

out = first_conv(x_30)
print("out-0", out.size())

time_emb = pos_emb[t]

for i, (block, down, time_block) in enumerate(zip(encoder_conv_blocks, down_conv_blocks, encoder_time_to_channel_blocks)):
    print("layer", i)
    time = time_block(time_emb)[:, :, None, None]
    print("time", time.size())
    out = out + time
    print("time-add", out.size())

    out = block(out)
    print("conv-out", out.size())

    out = down(out)
    print("strided-conv-out", out.size())
    print()

Même logique pour le décodeur :

In [ ]:
# decoder's layers definition

decoder_channels = [(c_o, c_i) for c_i, c_o in reversed(channels)]

up_conv_blocks = nn.ModuleList(
    StrideConvBlock(c_i, c_i, norm_groups, "up")
    for c_i, _ in decoder_channels
)

decoder_time_to_channels_blocks = nn.ModuleList(
    nn.Sequential(
        nn.Linear(TIME_EMBEDDING_SIZE, c_i),
        nn.Mish(),
        nn.Linear(c_i, c_i)
    )
    for c_i, _ in decoder_channels
)

decoder_conv_blocks = nn.ModuleList(
    nn.Sequential(
        ConvBlock(c_i, c_i, norm_groups),
        ConvBlock(c_i, c_o, norm_groups),
    )
    for c_i, c_o in decoder_channels
)

# on output des variables aléatoire d'une distribution normale
# => pas de limites théoriques => pas d'activation
last_block = nn.Conv2d(
    decoder_channels[-1][1],
    input_channels,
    kernel_size=(3, 3),
    stride=(1, 1),
    padding=(1, 1),
)

In [ ]:
# decoder inputs

out.size(), time_emb.size()

In [ ]:
# decoder forward

for i, (up, time_block, block) in enumerate(zip(up_conv_blocks, decoder_time_to_channels_blocks, decoder_conv_blocks)):
    print("layer", i)

    out = up(out)
    print("up", out.size())

    time_proj = time_block(time_emb)[:, :, None, None]
    print("time", time_proj.size())

    out = out + time_proj
    print("time-add", out.size())

    out = block(out)
    print("conv", out.size())
    print()

eps = last_block(out)
print("eps", eps.size())

### Dernière étape : les connexions "bypass" entre encodeur et décodeur

L'idée : récupérer l'image intermédiaire de chaque couche de l'encodeur et la concaténer (au niveau de l'axe des canaux) à l'image intermédiaire homologue coté décodeur.

![](https://miro.medium.com/v2/resize:fit:1400/1*VUS2cCaPB45wcHHFp_fQZQ.png)


Il nous faut modifier les blocks de convolutions du décodeur pour que ceux-ci prennent deux fois plus de canaux en entrée :

In [ ]:
decoder_conv_blocks = nn.ModuleList(
    nn.Sequential(
        ConvBlock(c_i * 2, c_i, norm_groups),
        ConvBlock(c_i, c_o, norm_groups),
    )
    for c_i, c_o in decoder_channels
)

In [ ]:
# et du coup le time embedding to channels aussi :

decoder_time_to_channels_blocks = nn.ModuleList(
    nn.Sequential(
        nn.Linear(TIME_EMBEDDING_SIZE, c_i * 2),
        nn.Mish(),
        nn.Linear(c_i * 2, c_i * 2)
    )
    for c_i, _ in decoder_channels
)

Le U-Net et sa fonction forward finie :

In [ ]:
out = first_conv(x_30)

time_emb = pos_emb[t]

bypasses = []

for i, (block, down, time_block) in enumerate(zip(encoder_conv_blocks, down_conv_blocks, encoder_time_to_channel_blocks)):
    time = time_block(time_emb)[:, :, None, None]
    out = out + time

    out = block(out)

    bypasses.append(out)

    out = down(out)

print(" ".join(str(bypass.size()) for bypass in bypasses))

for i, (up, time_block, block, bypass) in enumerate(zip(up_conv_blocks, decoder_time_to_channels_blocks, decoder_conv_blocks, reversed(bypasses))):
    out = up(out)

    # dim=1  =>  les canaux des pixels
    out = th.cat([out, bypass], dim=1)
    print("decoder", i, "size_1 :", out.size())

    time_proj = time_block(time_emb)[:, :, None, None]
    out = out + time_proj

    out = block(out)
    print("decoder", i, "size_2 :",out.size())

eps = last_block(out)
print(eps.size())

Plus qu'à mettre le tout en Module PyTorch et se consacrer à l'entrainement

## Entrainement

In [ ]:
from music_diffusion.networks import Noiser, Denoiser

In [ ]:
T = 100

noiser = Noiser(T, 1e-4, 2e-2)
denoiser = Denoiser(
    3, T, 16, 1e-4, 2e-2, [(8, 16), (16, 32), (32, 64)], 8
)

![](https://huggingface.co/blog/assets/78_annotated-diffusion/training.png)

In [ ]:
time_batch_size = 2

optim = th.optim.Adam(denoiser.parameters(), lr=1e-4)

In [ ]:
# training loop

# une seule image dans le batch
x_0 = to_tensor(Image.open("./190430171751-mona-lisa.jpg")).unsqueeze(0)
t = th.randint(0, T, (x_0.size(0), time_batch_size))

# diffusion / bruitage
x_t, eps = noiser(x_0, t)

# prediction du bruit
eps_theta, _ = denoiser(x_t, t)

print(x_t.size(), eps.size(), eps_theta.size())

# on cherche à prédire le bruit
loss = th.pow(eps - eps_theta, 2.)
loss = loss.mean()

# backward sur le réseau de neurone + MAJ des poids
optim.zero_grad()
loss.backward()
optim.step()

In [ ]:
next(denoiser.parameters()).grad

## Dernière étape : la génération

![](https://eugeneyan.com/assets/ddpm-sampling.jpg)

En code :

In [ ]:
input_channels = 3
with th.no_grad():
    denoiser.cuda()
    denoiser.eval()

    # on tire une image aléatoire : input pour générer la donnée
    x_t = th.randn(1, input_channels, *x_0.size()[2:], device="cuda")

    # les étapes de T - 1 à 0
    for t in tqdm(reversed(range(T))):
        z = th.randn_like(x_t, device="cuda") if t > 0 else th.zeros_like(x_t, device="cuda")

        # predire le bruit
        eps_theta, _ = denoiser(
            x_t.unsqueeze(1),
            th.tensor([[t]], device="cuda"),
        )
        eps_theta = eps_theta.squeeze(1)

        # moyenne
        mu = (x_t - eps_theta * (1 - alphas[t]) / th.sqrt(1 - alphas_cum_prod[t])) / th.sqrt(alphas[t])

        # variance
        var = betas[t]

        # création du x à l'étape t - 1
        x_t = mu + th.sqrt(var) * z

In [ ]:
x_t.size()

Voilà ! Vous maitrisez maintenant le papa des diffusions modèles !!

Petit souci : dans la pratique sur la musique ça ne fonctionnait pas au top, let's go voir ça dans un autre notebook !

![](https://wompampsupport.azureedge.net/fetchimage?siteId=7575&v=2&jpgQuality=100&width=700&url=https%3A%2F%2Fi.kym-cdn.com%2Fentries%2Ficons%2Ffacebook%2F000%2F029%2F405%2Fjordan.jpg)

:P